# Introduction:

Welcome to Samar Haider, Kaishu Mason, and Kyle Sullivan's CIS 545 Final Project for Spring 2022!

In this notebook, we will dive into the world of NCAA men's basketball. Every year, March Madness draws national attention as schools from across the country compete in a large multi-week tournament to crown college basketball's next champion. Perhaps the most significant athletic competition for data enthusiasts, these numerous high-stake elimination games between typically unfamiliar opponents present an excellent opportunity to build and test predictive sports models.

Our goal is to analyze college basketball data from 2003 to 2022 to produce a model that predicts matchup winners and losers, with the ultimate goal of accurately predicting the 2022 March Madness tournament.

# Install/Import the revelant libraries and packages.
We will be using Pandas, Pandas SQL, Geopandas, Geopy, and Plotly.

In [1]:
# !pip install pandas==1.1.5
# !pip install pandasql
!pip install SQLAlchemy==1.4.46


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import pandasql as ps
import numpy as np

# Import the Data

Connect to the google drive and download the necessary csv files.

In [3]:
# # Setup google drive:
# from google.colab import drive
# drive.mount("/content/gdrive")

We will be using the following datasets from Kaggle:


*   MRegularSeasonDetailedResults.csv
*   MConferenceTourneyGames.csv
*   MNCAATourneyDetailedResults.csv
*   MNCAATourneySeeds.csv
*   MTeamCoaches.csv
*   MTeams.csv
*   MTeamConferences.csv
*   Conferences.csv
*   MMasseyOrdinals_thruDay128.csv
*   MGameCities.csv
*   Cities.csv





In [4]:
# Read the data from Google Drive:
reg_season_detailed_df = pd.read_csv('/content/MRegularSeasonDetailedResults2023.csv')
conf_tourney_games_df = pd.read_csv('/content/MConferenceTourneyGames2023.csv')
tourney_games_detailed_df = pd.read_csv('/content/MNCAATourneyDetailedResults2023.csv')
tourney_seeds_df = pd.read_csv('/content/MNCAATourneySeeds2023.csv')
team_coaches_df = pd.read_csv('/content/MTeamCoaches2023.csv')
teams_df = pd.read_csv('/content/MTeams2023.csv')
team_conf_df = pd.read_csv('/content/MTeamConferences2023.csv')
conferences_df = pd.read_csv('/content/Conferences2023.csv')
massey_ordinals_df = pd.read_csv('/content/2023/MMasseyOrdinals_thruDay128_2023.csv')
game_cities_df = pd.read_csv('/content/MGameCities2023.csv')
cities_df = pd.read_csv('/content/Cities2023.csv')

In [5]:
team_conf_df

,Season,TeamID,ConfAbbrev
0,1985,1102,wac
1,1985,1103,ovc
2,1985,1104,sec
3,1985,1106,swac
4,1985,1108,swac
...,...,...,...
12657,2023,1473,ovc
12658,2023,1474,a_sun
12659,2023,1475,ovc
12660,2023,1476,nec


In [6]:
year = 2019

# Wrangling

## Combine the regular season and March Madness tournament game DataFrames

First, we want to gather our game data into a single DataFrame (combined_detailed_df). We'll accomplish this task by concatenating the March Madness tournament games (tourney_games_detailed_df) to the regular season games (reg_season_detailed_df). At the same time, let's insert 'ConfTourney' and 'MMTourney' columns into the result taking binary values of 0 or 1 to indicate if this game instance was a conference tournament game or a March Madness tournament game. Note that the regular season DataFrame (reg_season_detailed_df) already includes conference tournament games.

In [7]:
reg_season_detailed_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107629,2023,132,1104,82,1401,63,N,0,27,66,...,15,20,24,17,20,6,13,5,2,21
107630,2023,132,1272,75,1222,65,A,0,27,53,...,25,19,28,22,20,11,10,5,4,19
107631,2023,132,1343,74,1463,65,H,0,23,57,...,22,13,19,6,25,12,10,3,7,20
107632,2023,132,1345,67,1336,65,N,0,24,62,...,23,14,15,4,27,14,10,4,2,20


In [8]:
# Add an MMTourney column to reg_season_detailed_df that takes 1 or 0.
# Since reg_season_detailed_df doesn't contain any MMTourney games, all entries are set to 0.
reg_season_detailed_df['MMTourney'] = 0

# Add a 'ConfTourney' column to reg_season_detailed_df taking a 1 if the game was a conference tournament match or 0 if not.
# Note that reg_season_detailed_df already contains all of the conference tournament games.

# We accomplish this by left merging reg_season_detailed_df and conf_tourney_games_df with the indicator column set to True.
# Initially we use a temp_df to hold these changes, but once everything checks out, we'll finish by setting
#    reg_season_detailed_df to temp_df.
temp_df = reg_season_detailed_df.merge(conf_tourney_games_df.drop(columns=['ConfAbbrev']), how='left',
                                       on=['Season', 'DayNum', 'WTeamID', 'LTeamID'], indicator=True)

# Next, we check where the indicator column '_merge' equals 'both' – these are instances found in both
#    reg_season_detailed_df and conf_tourney_games_df. Using the apply method, set the 'ConfTourney' values
#    of these 'both' rows to 1, and all others to 0. Basically, if the game was in our conf_tourney_games_df,
#    its 'ConfTourney' value should be 1, and all other games should have a 0 here.
temp_df['ConfTourney'] = temp_df.apply(lambda row : 1 if row['_merge'] == 'both' else 0, axis=1)

# Remove the '_merge' column now that it is no longer needed.
temp_df = temp_df.drop(columns=['_merge'])

# Finish by setting reg_season_detailed_df equal to temp_df.
reg_season_detailed_df = temp_df

# Now, add 'ConfTourney' and 'MMTourney' columns to tourney_games_detailed_df.
# Since all of the games in tourney_games_detailed_df are exlusively MMTourney games, everything in 'ConfTourney' gets set to 0
#    and everything in 'MMTourney' gets set to 1.
tourney_games_detailed_df['ConfTourney'] = 0
tourney_games_detailed_df['MMTourney'] = 1


# To finish, let's combine (concatenate) all reg_season_detailed_df and tourney_games_detailed_df into combined_detailed_df.
frames = [reg_season_detailed_df, tourney_games_detailed_df]
combined_detailed_df = pd.concat(frames)
combined_detailed_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,MMTourney,ConfTourney
0,2003,10,1104,68,1328,62,N,0,27,58,...,22,10,22,8,18,9,2,20,0,0
1,2003,10,1272,70,1393,63,N,0,26,62,...,20,20,25,7,12,8,6,16,0,0
2,2003,11,1266,73,1437,61,N,0,24,58,...,23,31,22,9,12,2,5,23,0,0
3,2003,11,1296,56,1457,50,N,0,18,38,...,15,17,20,9,19,4,3,23,0,0
4,2003,11,1400,77,1208,71,N,0,30,61,...,27,21,15,12,10,7,1,14,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,2022,146,1242,76,1274,50,N,0,29,58,...,13,5,21,7,14,7,4,20,1,0
1244,2022,146,1314,69,1389,49,N,0,25,61,...,10,4,25,11,7,4,7,18,1,0
1245,2022,152,1242,81,1437,65,N,0,29,54,...,10,12,17,12,9,3,0,11,1,0
1246,2022,152,1314,81,1181,77,N,0,27,64,...,20,13,25,12,4,7,4,18,1,0


## Convert 'Winners' and 'Losers' to 'Team' and 'Opponent', such that the victors and defeated appear in each, effectively doubling our game instances while removing outcome indicative identifying columns

Now that we have all of our game instances in a single DataFrame, let's remove the winner and loser indicators.

Initially, games have winners (and all of their stats) in separate columns from losers (and all of their stats). However, we don't want this division. Instead, teams should appear in 'TeamID' and 'OppTeamID' columns, with their stats following a similar naming convention. Rather than arbitrarily deciding which team will be labeled the 'opponent', we will create two instances of every matchup, with each team appearing once as the 'team' and once as the 'opponent'. This will allow us to predict win/loss outcomes from the perspective of 'TeamID'. Without making these changes, we would be predicting outcomes from the perspective of either 'WTeamID'  or 'LTeamID' exclusively, which would produce a trivial always 'win' or always 'loss' prediction. Furthermore, this produces class parity, with half of our game labels being wins and the other half being losses.

In [9]:
# Create winners_combined_games_df from the winners in combined_detailed_df.

# Make a winners_combined_games_df from combined_detailed_df, with the columns reordered.
winners_combined_games_df = combined_detailed_df[['Season', 'DayNum', 'ConfTourney', 'MMTourney', 'WTeamID', 'LTeamID', 'WLoc', 'NumOT', 'WScore', 'LScore',
                                                  'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',
                                                  'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]

# Standardize the winners_combined_games_df columns by removing winner and loser naming conventions,
#    changing 'WTeamID' to 'TeamID' and 'LTeamID' to 'OppTeamID'. Do the same for their respective stats.
winners_combined_games_df = winners_combined_games_df.rename(columns={'WTeamID': 'TeamID', 'LTeamID': 'OppTeamID', 'WLoc': 'Loc', 'WScore': 'Score', 'LScore': 'OppScore',

                                                                      'WFGM': 'FGM', 'WFGA': 'FGA', 'WFGM3': 'FGM3', 'WFGA3': 'FGA3', 'WFTM': 'FTM', 'WFTA': 'FTA',
                                                                      'WOR': 'OR', 'WDR': 'DR', 'WAst': 'Ast', 'WTO': 'TO', 'WStl': 'Stl', 'WBlk': 'Blk', 'WPF': 'PF',

                                                                      'LFGM': 'OppFGM', 'LFGA': 'OppFGA', 'LFGM3': 'OppFGM3', 'LFGA3': 'OppFGA3', 'LFTM': 'OppFTM',
                                                                      'LFTA': 'OppFTA', 'LOR': 'OppOR', 'LDR': 'OppDR', 'LAst': 'OppAst', 'LTO': 'OppTO', 'LStl': 'OppStl',
                                                                      'LBlk': 'OppBlk', 'LPF': 'OppPF'
                                                                      })

# Create a 'ScoreDiff' column to hold the difference between TeamID's score and OppTeamID's score.
# These values will all be positive since TeamID currently only holds the winning team IDs.
winners_combined_games_df['ScoreDiff'] = winners_combined_games_df['Score'] - winners_combined_games_df['OppScore']

# Add an 'Outcome' column, which will have 'W' values signifying that TeamID won. Note that this could also be infered from
#    the positive score difference. 'W' and 'L' Outcomes can easily be converted to 1 and 0 later for ML applications.
winners_combined_games_df['Outcome'] = 'W'

In [10]:
# Create losers_combined_games_df from the losers in combined_detailed_df.

# Make a losers_combined_games_df from combined_detailed_df, with the columns reordered.
losers_combined_games_df = combined_detailed_df[['Season', 'DayNum', 'ConfTourney', 'MMTourney', 'WTeamID', 'LTeamID', 'WLoc', 'NumOT', 'WScore', 'LScore',
                                                  'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',
                                                  'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF']]


# Standardize the losers_combined_games_df columns by removing winner and loser naming conventions,
#    changing 'LTeamID' to 'TeamID' and 'WTeamID' to 'OppTeamID'. Do the same for their respective stats.
losers_combined_games_df = losers_combined_games_df.rename(columns={'WTeamID': 'OppTeamID', 'LTeamID': 'TeamID', 'WLoc': 'Loc', 'WScore': 'OppScore', 'LScore': 'Score',

                                                                    'WFGM': 'OppFGM', 'WFGA': 'OppFGA', 'WFGM3': 'OppFGM3', 'WFGA3': 'OppFGA3', 'WFTM': 'OppFTM',
                                                                    'WFTA': 'OppFTA', 'WOR': 'OppOR', 'WDR': 'OppDR', 'WAst': 'OppAst', 'WTO': 'OppTO', 'WStl': 'OppStl',
                                                                    'WBlk': 'OppBlk', 'WPF': 'OppPF',

                                                                    'LFGM': 'FGM', 'LFGA': 'FGA', 'LFGM3': 'FGM3', 'LFGA3': 'FGA3', 'LFTM': 'FTM', 'LFTA': 'FTA',
                                                                    'LOR': 'OR', 'LDR': 'DR', 'LAst': 'Ast', 'LTO': 'TO', 'LStl': 'Stl', 'LBlk': 'Blk', 'LPF': 'PF'
                                                                    })

# Create a 'ScoreDiff' column to hold the difference between TeamID's score and OppTeamID's score.
# These values will all be negative since 'TeamID' currently only holds the losing team IDs.
losers_combined_games_df['ScoreDiff'] = losers_combined_games_df['Score'] - losers_combined_games_df['OppScore']

# Add an 'Outcome' column, which will have 'L' values signifying that TeamID lost. Note that this could also be infered from
#    the negative score difference. 'W' and 'L' Outcomes can easily be converted to 1 and 0 later for ML applications.
losers_combined_games_df['Outcome'] = 'L'

Now that we have all of our game instances in a single DataFrame, let's remove the winner and loser indicators.

Since our dataset initially had the location stored as 'WLoc' from the perspective of WTeamID, we need to adjust our new losers_combined_games_df to show the new 'Loc' values from their flipped perspective. For example, home games ('H') should read as Away games ('A'), and vice versa. Neutral games ('N') should remain the same.

In [11]:
# This function will convert the home/away status from the winner's perspective to the loser's perspective.
# This will be used to convert the 'Loc' column in the losers_combined_games_df.
def loser_home_away_neutral_locator(value):
  if value == 'H':
    return 'A'
  elif value == 'A':
    return 'H'
  elif value == 'N':
    return 'N'

In [12]:
# First, make a copy of the original losers_combined_games_df pre-alteration.
# We want to compare our results against this to make sure everything worked as expected.
orig_losers_combined_games_df = losers_combined_games_df.copy()

# Next, convert the values in the 'Loc' column using our previously defined loser_home_away_neutral_locator function.
losers_combined_games_df['Loc'] = orig_losers_combined_games_df['Loc'].apply(loser_home_away_neutral_locator)

In [13]:
# Let's check Loc before the conversion.
orig_losers_combined_games_df = orig_losers_combined_games_df.sort_values('TeamID')
orig_losers_combined_games_df

,Season,DayNum,ConfTourney,MMTourney,OppTeamID,TeamID,Loc,NumOT,OppScore,Score,...,FTA,OR,DR,Ast,TO,Stl,Blk,PF,ScoreDiff,Outcome
79962,2018,86,0,0,1394,1101,H,0,80,66,...,30,8,17,12,19,11,0,22,-14,L
92506,2020,120,0,0,1372,1101,A,0,77,72,...,28,13,25,9,16,7,4,29,-5,L
73933,2017,73,0,0,1368,1101,A,0,56,50,...,25,6,22,8,19,7,2,27,-6,L
59150,2014,101,0,0,1230,1101,H,0,80,68,...,21,6,19,9,14,6,0,19,-12,L
60832,2015,16,0,0,1409,1101,H,0,65,39,...,7,13,17,12,13,9,2,22,-26,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107219,2023,121,0,0,1394,1477,A,1,93,88,...,18,4,22,16,9,4,5,18,-5,L
104491,2023,66,0,0,1311,1477,A,0,66,63,...,14,6,23,16,15,7,4,17,-3,L
107418,2023,125,1,0,1270,1477,N,0,79,78,...,20,8,19,16,7,5,3,17,-1,L
102613,2023,19,0,0,1209,1477,H,0,57,53,...,4,8,17,11,12,6,2,14,-4,L


In [14]:
# Now, ceck it after the conversion. As expected, the values were successfully converted.
losers_combined_games_df = losers_combined_games_df.sort_values('TeamID')
losers_combined_games_df

,Season,DayNum,ConfTourney,MMTourney,OppTeamID,TeamID,Loc,NumOT,OppScore,Score,...,FTA,OR,DR,Ast,TO,Stl,Blk,PF,ScoreDiff,Outcome
79962,2018,86,0,0,1394,1101,A,0,80,66,...,30,8,17,12,19,11,0,22,-14,L
92506,2020,120,0,0,1372,1101,H,0,77,72,...,28,13,25,9,16,7,4,29,-5,L
73933,2017,73,0,0,1368,1101,H,0,56,50,...,25,6,22,8,19,7,2,27,-6,L
59150,2014,101,0,0,1230,1101,A,0,80,68,...,21,6,19,9,14,6,0,19,-12,L
60832,2015,16,0,0,1409,1101,A,0,65,39,...,7,13,17,12,13,9,2,22,-26,L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107219,2023,121,0,0,1394,1477,H,1,93,88,...,18,4,22,16,9,4,5,18,-5,L
104491,2023,66,0,0,1311,1477,H,0,66,63,...,14,6,23,16,15,7,4,17,-3,L
107418,2023,125,1,0,1270,1477,N,0,79,78,...,20,8,19,16,7,5,3,17,-1,L
102613,2023,19,0,0,1209,1477,A,0,57,53,...,4,8,17,11,12,6,2,14,-4,L


In [15]:
# Finally, let's concatenate winners_reg_season_games_df and losers_reg_season_games_df into a single
#    combined_games_df DataFrame with twice as many game instances as combined_games_df had initially.
#    We will update combined_games_df, setting it to the result of this concatenation.
frames = [winners_combined_games_df, losers_combined_games_df]
combined_games_df = pd.concat(frames)
combined_games_df

,Season,DayNum,ConfTourney,MMTourney,TeamID,OppTeamID,Loc,NumOT,Score,OppScore,...,OppFTA,OppOR,OppDR,OppAst,OppTO,OppStl,OppBlk,OppPF,ScoreDiff,Outcome
0,2003,10,0,0,1104,1328,N,0,68,62,...,22,10,22,8,18,9,2,20,6,W
1,2003,10,0,0,1272,1393,N,0,70,63,...,20,20,25,7,12,8,6,16,7,W
2,2003,11,0,0,1266,1437,N,0,73,61,...,23,31,22,9,12,2,5,23,12,W
3,2003,11,0,0,1296,1457,N,0,56,50,...,15,17,20,9,19,4,3,23,6,W
4,2003,11,0,0,1400,1208,N,0,77,71,...,27,21,15,12,10,7,1,14,6,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107219,2023,121,0,0,1477,1394,H,1,88,93,...,23,9,32,17,10,6,1,18,-5,L
104491,2023,66,0,0,1477,1311,H,0,63,66,...,13,6,22,11,11,6,2,12,-3,L
107418,2023,125,1,0,1477,1270,N,0,78,79,...,23,10,25,14,8,3,0,14,-1,L
102613,2023,19,0,0,1477,1209,A,0,53,57,...,14,6,21,8,12,6,2,11,-4,L


## Add Team and Opponent Rankings

Now that we've wrangled our games into a single DataFrame, let's add more information. We'll start by supplying a ranking metric to both the teams and their opponents.

We'll use the Massey Ordinals from massey_ordinals_limited_df, a Kaggle dataset containing team rankings by season and day number from various ranker sources (i.e., the AP, USA Today Coaches, etc.). However, many of these rankers do not exist for the entire scope of our data (2003 to 2022). Furthermore, some of these from earlier years have suspiciously irregular rankings, suggesting that they may only be ranking teams in a specific conference. We don't want the 3rd best Ivy League team to take on an overall ranking of 3 in one of the ranking systems feeding into our average.

We'll remove these issues by only looking at ranking sources that are still around in the most recent 2022 season (sources still producing rankings). Furthermore, let's conservatively check if any of those remaining rankers limited their rankings to fewer than 100 teams. Rankers with far fewer than this may only be looking at specific conferences.

After settling on a group of rankers, let's average a team's season/day-specific rankings from our narrowed ranker sources and merge these into our combined_games_df.

In [16]:
# Create a set of all of the rankers in massey_ordinals_df that have 2022 season rankings.
massey_systems = set(massey_ordinals_df[massey_ordinals_df['Season'] == 2022]['SystemName'])
# Make a new massey_ordinals_limited_df containing only those from our massey_systems set.
massey_ordinals_limited_df = massey_ordinals_df[massey_ordinals_df['SystemName'].isin(massey_systems)]

# Investigate any rankers with seasons containing fewers than 100 ranking slots
#    (systems that didn't include the top 100 teams – maybe just the top 20).

# Three rankers appeared in all seasons. The AP, USA Today Coaches, and Mike DeSimone's Rankings.
# However, these are all emcompassing rankers. None focus on a single conference.
# Although they do not have as many rankings (25 and 26), they should still be included,
#    especially since AP and USA are two of the most watched sources. Teams outside of the top 25/26
#    will take 'NaN' values from these rankers, which won't impact their average ranking in out calculation.
temp_df = massey_ordinals_limited_df.groupby(['Season', 'SystemName'])[['OrdinalRank']].max().reset_index()
temp_df[temp_df['OrdinalRank'] < 100]

,Season,SystemName,OrdinalRank
0,2003,AP,25
12,2003,USA,25
16,2004,AP,25
20,2004,DES,25
28,2004,USA,25
...,...,...,...
697,2022,USA,25
704,2023,AP,25
716,2023,DES,25
723,2023,ESR,50


The issue with this process is that the days in massey_ordinals_limited_df only correspond to days when a new ranking was announced for a given team. Many of these days fail to match with instances in combined_games_df during a merge since they only do so when teams play on the same day their ranking updates.

The solution is to perform an outer merge followed by a team and season-specific forward fill across its 'NaN' 'Rank' column values. Additionally, we must sort by 'Season', 'DayNum', and 'TeamID' before forward filling to ensure that we maintain the correct order when padding forward our values. Finally, we will perform a similar backward fill to clean up any missing data earlier in the year, using the first available ranking. This method seems far superior to just imputing each team's seasonal averages.

In [17]:
# Here, we average our massey_ordinals_limited_df by 'Season', 'RankingDayNum' (renamed 'DayNum'),
#    and 'TeamID'.
ranking_query = """
SELECT Season, TeamID, RankingDayNum AS 'DayNum', AVG(OrdinalRank) AS 'Rank'
FROM massey_ordinals_limited_df m
GROUP BY Season, RankingDayNum, TeamID
"""

avg_massey_ordinals_limited_df = ps.sqldf(ranking_query, locals())
avg_massey_ordinals_limited_df

,Season,TeamID,DayNum,Rank
0,2003,1102,35,159.000000
1,2003,1103,35,229.000000
2,2003,1104,35,12.000000
3,2003,1105,35,314.000000
4,2003,1106,35,260.000000
...,...,...,...,...
217008,2023,1473,128,339.964912
217009,2023,1474,128,206.754386
217010,2023,1475,128,272.807018
217011,2023,1476,128,315.140351


In [18]:
# Perform an outer merge between combined_games_df and avg_massey_ordinals_limited_df to bring the rankings into our games DataFrame.
# We want all instances, even those that don't match up on 'DayNum'.
combined_games_df = combined_games_df.merge(avg_massey_ordinals_limited_df, on=['Season', 'TeamID', 'DayNum'], how='outer')

In [19]:
# Let's forward fill our 'NaN' values for each team and season.

# First, make a set of all team_ids.
team_ids = set(combined_games_df['TeamID'].tolist())

# Next sort combined_games_df by 'Season', 'DayNum', and 'TeamID'
combined_games_df = combined_games_df.sort_values(['Season', 'DayNum', 'TeamID']).reset_index()

# Loop through all seasons and teams.
for season in range(2003, 2023):
  for team in team_ids:

    # Each time, forward fill the Rank values (we can include 'Season' and 'DayNum' here because they are never 'NaN'),
    #    backfill early 'NaN' entries, and save the resulting DataFrame to temp_df.
    temp_df = combined_games_df[(combined_games_df['Season'] == season) & (combined_games_df['TeamID'] == team)] \
    [['Season', 'DayNum', 'Rank']].ffill().bfill()

    # Get the indices from temp_df
    indices = temp_df.index.tolist()

    # Set the 'Rank' values for these specific rows in combined_games_df to our new
    #    forward filled 'Rank' values in temp_df.
    combined_games_df.loc[indices, 'Rank'] = temp_df['Rank']

# Drop the 'NaN' values based on the 'OppTeamID' column to remove instances from avg_massey_ordinals_limited_df
#    that didn't matchup with game instances from combined_games_df during the merge.
combined_games_df = combined_games_df.dropna(subset=['OppTeamID'])

Test the results with The University of Florida (TeamID 1196). Expanding the DataFrame below, we see that the 'Rank' column is correctly forward-filled and backward-filled. Of course, some teams are never ranked during a season (if they aren't good enough), so we still see 'NaN' values in combined_games_df.

In [20]:
combined_games_df[combined_games_df['TeamID'] == 1196][['Season', 'DayNum', 'TeamID', 'OppTeamID', 'Rank', 'Outcome']]

,Season,DayNum,TeamID,OppTeamID,Rank,Outcome
53,2003,15,1196,1256.0,20.000000,W
66,2003,17,1196,1183.0,20.000000,W
350,2003,20,1196,1157.0,20.000000,W
607,2003,23,1196,1390.0,20.000000,L
686,2003,25,1196,1242.0,20.000000,W
...,...,...,...,...,...,...
388966,2023,114,1196,1246.0,63.481481,L
389470,2023,117,1196,1435.0,NaN,L
389773,2023,120,1196,1208.0,NaN,W
390409,2023,124,1196,1261.0,NaN,W


In [21]:
combined_games_df

,index,Season,DayNum,ConfTourney,MMTourney,TeamID,OppTeamID,Loc,NumOT,Score,...,OppOR,OppDR,OppAst,OppTO,OppStl,OppBlk,OppPF,ScoreDiff,Outcome,Rank
0,0,2003,10,0.0,0.0,1104,1328.0,N,0.0,68.0,...,10.0,22.0,8.0,18.0,9.0,2.0,20.0,6.0,W,12.0
1,1,2003,10,0.0,0.0,1272,1393.0,N,0.0,70.0,...,20.0,25.0,7.0,12.0,8.0,6.0,16.0,7.0,W,67.0
2,176505,2003,10,0.0,0.0,1328,1104.0,N,0.0,62.0,...,14.0,24.0,13.0,23.0,7.0,1.0,22.0,-6.0,L,24.0
3,196796,2003,10,0.0,0.0,1393,1272.0,N,0.0,63.0,...,15.0,28.0,16.0,13.0,4.0,4.0,18.0,-7.0,L,120.0
4,134437,2003,11,0.0,0.0,1186,1458.0,A,0.0,55.0,...,12.0,24.0,12.0,9.0,9.0,3.0,18.0,-26.0,L,123.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391291,107631,2023,132,1.0,0.0,1343,1463.0,H,0.0,74.0,...,6.0,25.0,12.0,10.0,3.0,7.0,20.0,9.0,W,NaN
391292,107632,2023,132,1.0,0.0,1345,1336.0,N,0.0,67.0,...,4.0,27.0,14.0,10.0,4.0,2.0,20.0,2.0,W,NaN
391293,199116,2023,132,1.0,0.0,1401,1104.0,N,0.0,63.0,...,21.0,29.0,16.0,12.0,6.0,7.0,19.0,-19.0,L,NaN
391294,107633,2023,132,1.0,0.0,1433,1173.0,N,0.0,68.0,...,16.0,22.0,11.0,8.0,1.0,6.0,21.0,12.0,W,NaN


In [22]:
# Merge combined_games_df with itself to get the opponent rankings for each game.

merge_query = """
SELECT t.*, p.Rank AS 'OppRank'
FROM combined_games_df t
JOIN combined_games_df p
ON t.OppTeamID=p.TeamID AND t.TeamID=p.OppTeamID AND t.Season=p.Season AND t.DayNum=p.DayNum
"""

combined_games_df = ps.sqldf(merge_query, locals())
combined_games_df

,index,Season,DayNum,ConfTourney,MMTourney,TeamID,OppTeamID,Loc,NumOT,Score,...,OppDR,OppAst,OppTO,OppStl,OppBlk,OppPF,ScoreDiff,Outcome,Rank,OppRank
0,0,2003,10,0.0,0.0,1104,1328.0,N,0.0,68.0,...,22.0,8.0,18.0,9.0,2.0,20.0,6.0,W,12.0,24.0
1,1,2003,10,0.0,0.0,1272,1393.0,N,0.0,70.0,...,25.0,7.0,12.0,8.0,6.0,16.0,7.0,W,67.0,120.0
2,176505,2003,10,0.0,0.0,1328,1104.0,N,0.0,62.0,...,24.0,13.0,23.0,7.0,1.0,22.0,-6.0,L,24.0,12.0
3,196796,2003,10,0.0,0.0,1393,1272.0,N,0.0,63.0,...,28.0,16.0,13.0,4.0,4.0,18.0,-7.0,L,120.0,67.0
4,134437,2003,11,0.0,0.0,1186,1458.0,A,0.0,55.0,...,24.0,12.0,9.0,9.0,3.0,18.0,-26.0,L,123.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217759,107631,2023,132,1.0,0.0,1343,1463.0,H,0.0,74.0,...,25.0,12.0,10.0,3.0,7.0,20.0,9.0,W,NaN,NaN
217760,107632,2023,132,1.0,0.0,1345,1336.0,N,0.0,67.0,...,27.0,14.0,10.0,4.0,2.0,20.0,2.0,W,NaN,NaN
217761,199116,2023,132,1.0,0.0,1401,1104.0,N,0.0,63.0,...,29.0,16.0,12.0,6.0,7.0,19.0,-19.0,L,NaN,NaN
217762,107633,2023,132,1.0,0.0,1433,1173.0,N,0.0,68.0,...,22.0,11.0,8.0,1.0,6.0,21.0,12.0,W,NaN,NaN


## Calculate Wins, Losses, and Win%

Now that we have all of our games and rankings together let's begin aggregating statistics. First, we will calculate the wins, losses, and win% for each team and season.

In [23]:
# Calculate the regular season wins by 'Season' and 'TeamID' and call this new DataFrame reg_season_wins_df.

# We can get the wins data from winners_combined_games_df. Just be sure to filter for regular season games ('MMTourney' == 0).
reg_season_wins_df = winners_combined_games_df[winners_combined_games_df['MMTourney'] == 0]
# With our new reg_season_wins_df, lets count how many wins each team had in a given season.
reg_season_wins_df = reg_season_wins_df.groupby(['Season', 'TeamID'])[['Score']].count().reset_index()
# Rename our count column to 'Wins' and sort by 'Season', 'Wins', and 'TeamID' in descending order.
reg_season_wins_df = reg_season_wins_df.rename(columns={'Score': 'Wins'}).sort_values(by=['Season', 'Wins', 'TeamID'], ascending=[False, False, False])

# Follow the same process, but with losers_combined_games_df to get the losses for each team and season.
reg_season_losses_df = losers_combined_games_df[losers_combined_games_df['MMTourney'] == 0]
reg_season_losses_df = reg_season_losses_df.groupby(['Season', 'TeamID'])[['Score']].count().reset_index()
reg_season_losses_df = reg_season_losses_df.rename(columns={'Score': 'Losses'}).sort_values(by=['Season', 'Losses', 'TeamID'], ascending=[False, True, False])

# Outer merge reg_season_wins_df and reg_season_losses_df into reg_season_wins_and_losses_df.
# We use an outer merge in case some teams went undefeated in a given year, or worse... went winless.
reg_season_wins_and_losses_df = reg_season_wins_df.merge(reg_season_losses_df, how='outer', on=['Season', 'TeamID']) \
                                                         .sort_values(by=['Season', 'Wins', 'Losses', 'TeamID'], ascending=[False, False, True, False])

# Fill 'NaN' values in 'Wins' and 'Losses' columns with 0.
reg_season_wins_and_losses_df = reg_season_wins_and_losses_df.fillna({"Wins": 0, "Losses": 0})

# Add a Win% column.
reg_season_wins_and_losses_df['Win%'] = (reg_season_wins_and_losses_df['Wins'] /
                                        (reg_season_wins_and_losses_df['Wins'] + reg_season_wins_and_losses_df['Losses'])) * 100

reg_season_wins_and_losses_df

,Season,TeamID,Wins,Losses,Win%
0,2023,1222,31.0,3.0,91.176471
1,2023,1158,30.0,3.0,90.909091
4,2023,1194,29.0,3.0,90.625000
2,2023,1417,29.0,5.0,85.294118
3,2023,1345,29.0,5.0,85.294118
...,...,...,...,...,...
7242,2003,1311,2.0,25.0,7.407407
7244,2003,1162,2.0,25.0,7.407407
7243,2003,1200,2.0,26.0,7.142857
7245,2003,1398,1.0,24.0,4.000000


Let's merge combined_games_df with our new reg_season_wins_and_losses_df twice to get the total season Wins, Losses, OppWins, OppLosses, Win%, and OppWin% for TeamID and OppTeamID. These values cover the entire season, giving a more extensive overview of each team's performance outside of the scope of one game. We considered calculating a cumulative win percentage but feared that this would introduce too much noise early in the season.

In [24]:
combined_games_df

,index,Season,DayNum,ConfTourney,MMTourney,TeamID,OppTeamID,Loc,NumOT,Score,...,OppDR,OppAst,OppTO,OppStl,OppBlk,OppPF,ScoreDiff,Outcome,Rank,OppRank
0,0,2003,10,0.0,0.0,1104,1328.0,N,0.0,68.0,...,22.0,8.0,18.0,9.0,2.0,20.0,6.0,W,12.0,24.0
1,1,2003,10,0.0,0.0,1272,1393.0,N,0.0,70.0,...,25.0,7.0,12.0,8.0,6.0,16.0,7.0,W,67.0,120.0
2,176505,2003,10,0.0,0.0,1328,1104.0,N,0.0,62.0,...,24.0,13.0,23.0,7.0,1.0,22.0,-6.0,L,24.0,12.0
3,196796,2003,10,0.0,0.0,1393,1272.0,N,0.0,63.0,...,28.0,16.0,13.0,4.0,4.0,18.0,-7.0,L,120.0,67.0
4,134437,2003,11,0.0,0.0,1186,1458.0,A,0.0,55.0,...,24.0,12.0,9.0,9.0,3.0,18.0,-26.0,L,123.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217759,107631,2023,132,1.0,0.0,1343,1463.0,H,0.0,74.0,...,25.0,12.0,10.0,3.0,7.0,20.0,9.0,W,NaN,NaN
217760,107632,2023,132,1.0,0.0,1345,1336.0,N,0.0,67.0,...,27.0,14.0,10.0,4.0,2.0,20.0,2.0,W,NaN,NaN
217761,199116,2023,132,1.0,0.0,1401,1104.0,N,0.0,63.0,...,29.0,16.0,12.0,6.0,7.0,19.0,-19.0,L,NaN,NaN
217762,107633,2023,132,1.0,0.0,1433,1173.0,N,0.0,68.0,...,22.0,11.0,8.0,1.0,6.0,21.0,12.0,W,NaN,NaN


In [25]:
# Merge combined_games_df and reg_season_wins_and_losses_df twice (hence the subquery).
# The first query will get TeamID's stats and the second will get the OppTeamID's stats.

merge_query = """
WITH

temp_df AS
(
  SELECT c.*, wl.Wins, wl.Losses, wl.'Win%'
  FROM combined_games_df c
  JOIN reg_season_wins_and_losses_df wl
  ON c.Season=wl.Season AND c.TeamID=wl.TeamID
)

SELECT t.*, wl.Wins AS 'OppWins', wl.Losses AS 'OppLosses', wl.'Win%' AS 'OppWin%'
FROM temp_df t
JOIN reg_season_wins_and_losses_df wl
ON t.Season=wl.Season AND t.OppTeamID=wl.TeamID
"""

combined_games_df = ps.sqldf(merge_query, locals())
combined_games_df

,index,Season,DayNum,ConfTourney,MMTourney,TeamID,OppTeamID,Loc,NumOT,Score,...,ScoreDiff,Outcome,Rank,OppRank,Wins,Losses,Win%,OppWins,OppLosses,OppWin%
0,0,2003,10,0.0,0.0,1104,1328.0,N,0.0,68.0,...,6.0,W,12.0,24.0,17.0,11.0,60.714286,24.0,6.0,80.000000
1,1,2003,10,0.0,0.0,1272,1393.0,N,0.0,70.0,...,7.0,W,67.0,120.0,23.0,6.0,79.310345,24.0,5.0,82.758621
2,176505,2003,10,0.0,0.0,1328,1104.0,N,0.0,62.0,...,-6.0,L,24.0,12.0,24.0,6.0,80.000000,17.0,11.0,60.714286
3,196796,2003,10,0.0,0.0,1393,1272.0,N,0.0,63.0,...,-7.0,L,120.0,67.0,24.0,5.0,82.758621,23.0,6.0,79.310345
4,134437,2003,11,0.0,0.0,1186,1458.0,A,0.0,55.0,...,-26.0,L,123.0,16.0,17.0,12.0,58.620690,22.0,7.0,75.862069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217759,107631,2023,132,1.0,0.0,1343,1463.0,H,0.0,74.0,...,9.0,W,NaN,NaN,19.0,8.0,70.370370,19.0,8.0,70.370370
217760,107632,2023,132,1.0,0.0,1345,1336.0,N,0.0,67.0,...,2.0,W,NaN,NaN,29.0,5.0,85.294118,22.0,13.0,62.857143
217761,199116,2023,132,1.0,0.0,1401,1104.0,N,0.0,63.0,...,-19.0,L,NaN,NaN,25.0,9.0,73.529412,29.0,5.0,85.294118
217762,107633,2023,132,1.0,0.0,1433,1173.0,N,0.0,68.0,...,12.0,W,NaN,NaN,27.0,7.0,79.411765,22.0,12.0,64.705882


## Calculating Single Game Stats

Get the average statistics for each game. For example, we want each team's:

*  AdjScore: adjusted score (per 40 minutes)
*  FG%: field goal percentage
*  FG3%: 3 point field goal percentage
*  FT%: free throw percentage
*  Ast/TO: assist to turnover ratio
*  NetTO: net turnovers (turnovers - steals)
*  Reb: total rebounds (offensive + defensive)

The opponent will have the same metrics but with an 'Opp' prefix.

In [26]:
# Calculate adjusted score per 40 minutes.
combined_games_df['AdjScore'] = combined_games_df['Score'] * (40 / (40 + 5 * combined_games_df['NumOT']))

# Calculate field goal percentage (this includes 2 point shots and 3 point shots).
combined_games_df['FG%'] = (combined_games_df['FGM'] / combined_games_df['FGA']) * 100

# Calculate 3 point field goal percentage.
combined_games_df['FG3%'] = (combined_games_df['FGM3'] / combined_games_df['FGA3']) * 100

# Calculate free throw percentage.
combined_games_df['FT%'] = (combined_games_df['FTM'] / combined_games_df['FTA']) * 100

# Calculate assit to turnover ratio.
combined_games_df['Ast/TO'] = combined_games_df['Ast'] / combined_games_df['TO']

# Calculate net turnovers (turnovers - steals).
combined_games_df['NetTO'] = combined_games_df['TO'] - combined_games_df['Stl']

# Calculate total rebounds as offensive rebounds + defensive rebounds.
combined_games_df['Reb'] = combined_games_df['OR'] + combined_games_df['DR']



# Normalize opponent's score over 40 minutes.
combined_games_df['OppAdjScore'] = combined_games_df['OppScore'] * (40 / (40 + 5 * combined_games_df['NumOT']))

# Calculate field goal percentage (this includes 2 point shots and 3 point shots).
combined_games_df['OppFG%'] = (combined_games_df['OppFGM'] / combined_games_df['OppFGA']) * 100

# Calculate 3 point field goal percentage.
combined_games_df['OppFG3%'] = (combined_games_df['OppFGM3'] / combined_games_df['OppFGA3']) * 100

# Calculate free throw percentage.
combined_games_df['OppFT%'] = (combined_games_df['OppFTM'] / combined_games_df['OppFTA']) * 100

# Calculate assit to turnover ratio.
combined_games_df['OppAst/TO'] = combined_games_df['OppAst'] / combined_games_df['OppTO']

# Calculate net turnovers (turnovers - steals).
combined_games_df['OppNetTO'] = combined_games_df['OppTO'] - combined_games_df['OppStl']

# Calculate total rebounds as offensive rebounds + defensive rebounds.
combined_games_df['OppReb'] = combined_games_df['OppOR'] + combined_games_df['OppDR']
combined_games_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217764 entries, 0 to 217763
Data columns (total 61 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   index        217764 non-null  int64  
 1   Season       217764 non-null  int64  
 2   DayNum       217764 non-null  int64  
 3   ConfTourney  217764 non-null  float64
 4   MMTourney    217764 non-null  float64
 5   TeamID       217764 non-null  int64  
 6   OppTeamID    217764 non-null  float64
 7   Loc          217764 non-null  object 
 8   NumOT        217764 non-null  float64
 9   Score        217764 non-null  float64
 10  OppScore     217764 non-null  float64
 11  FGM          217764 non-null  float64
 12  FGA          217764 non-null  float64
 13  FGM3         217764 non-null  float64
 14  FGA3         217764 non-null  float64
 15  FTM          217764 non-null  float64
 16  FTA          217764 non-null  float64
 17  OR           217764 non-null  float64
 18  DR           217764 non-

In [27]:
teams_df[['TeamID',	'TeamName']].rename(columns={'TeamID':'OppTeamID', 'TeamName':'OppTeamName'})

,OppTeamID,OppTeamName
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M
...,...,...
372,1473,Lindenwood
373,1474,Queens NC
374,1475,Southern Indiana
375,1476,Stonehill


In [28]:
combined_games_df = pd.merge(left=teams_df[['TeamID',	'TeamName']], right=combined_games_df, on=['TeamID'])
combined_games_df = pd.merge(left=teams_df[['TeamID',	'TeamName']].rename(columns={'TeamID':'OppTeamID', 'TeamName':'OppTeamName'}), right=combined_games_df, on=['OppTeamID'])
combined_games_df

,OppTeamID,OppTeamName,TeamID,TeamName,index,Season,DayNum,ConfTourney,MMTourney,Loc,...,Ast/TO,NetTO,Reb,OppAdjScore,OppFG%,OppFG3%,OppFT%,OppAst/TO,OppNetTO,OppReb
0,1101,Abilene Chr,1102,Air Force,109073,2018,37,0.0,0.0,H,...,0.750000,15.0,33.0,62.000000,42.592593,36.000000,46.666667,0.846154,5.0,24.0
1,1101,Abilene Chr,1115,Ark Pine Bluff,113042,2015,50,0.0,0.0,N,...,0.357143,11.0,30.0,69.000000,53.333333,50.000000,68.750000,0.833333,11.0,29.0
2,1101,Abilene Chr,1116,Arkansas,93728,2021,50,0.0,0.0,H,...,1.153846,7.0,36.0,72.000000,47.540984,22.727273,60.000000,1.083333,9.0,25.0
3,1101,Abilene Chr,1117,Arkansas St,76814,2018,14,0.0,0.0,H,...,0.600000,8.0,33.0,69.000000,51.020408,17.647059,80.000000,0.785714,10.0,21.0
4,1101,Abilene Chr,1117,Arkansas St,113826,2019,4,0.0,0.0,A,...,0.812500,10.0,29.0,94.000000,56.363636,57.142857,77.419355,0.944444,10.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217759,1477,TX A&M Commerce,1394,TAM C. Christi,196900,2023,115,0.0,0.0,H,...,1.400000,4.0,26.0,84.000000,63.265306,35.000000,78.947368,1.062500,11.0,26.0
217760,1477,TX A&M Commerce,1394,TAM C. Christi,107219,2023,121,0.0,0.0,A,...,1.700000,4.0,41.0,78.222222,48.437500,37.142857,72.222222,1.777778,5.0,26.0
217761,1477,TX A&M Commerce,1400,Texas,104080,2023,57,0.0,0.0,H,...,3.111111,2.0,31.0,72.000000,53.191489,46.666667,83.333333,0.733333,8.0,20.0
217762,1477,TX A&M Commerce,1421,UNC Asheville,102583,2023,18,0.0,0.0,N,...,0.923077,7.0,39.0,64.000000,37.313433,25.000000,58.333333,1.222222,2.0,37.0


In [29]:
combined_year_df = combined_games_df[combined_games_df['Season'] == year]
combined_year_df = combined_year_df.drop(columns='index')
combined_year_df['NeutralSite'] = np.where(combined_year_df['Loc'] == 'N', 1, 0)

In [31]:
team1_df = combined_year_df.loc[:,:'PF']
temp_df = combined_year_df.loc[:,'OppTeamID':'OppScore']
temp2_df = combined_year_df.loc[:,'OppFGM':'OppPF']
team1_df = team1_df.drop(columns=['OppScore'])

team2_df = pd.concat([temp_df, temp2_df], axis=1)
team2_df = team2_df.drop(columns=['Score'])

In [32]:
for col in team2_df.columns:
  if col != 'OppTeamID' and col != 'OppTeamName':
    new_col = col.replace('Opp','')
    team2_df = team2_df.rename(columns={col:new_col})

In [33]:
team2_df = team2_df.rename(columns={'TeamName':'temp', 'OppTeamName':'TeamName'})
team2_df = team2_df.rename(columns={'temp':'OppTeamName'})

team2_df = team2_df.rename(columns={'TeamID':'temp', 'OppTeamID':'TeamID'})
team2_df = team2_df.rename(columns={'temp':'OppTeamID'})
team2_df

,TeamID,TeamName,OppTeamID,OppTeamName,Season,DayNum,ConfTourney,MMTourney,Loc,NumOT,...,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF
4,1101,Abilene Chr,1117,Arkansas St,2019,4,0.0,0.0,A,0.0,...,14.0,24.0,31.0,12.0,28.0,17.0,18.0,8.0,0.0,27.0
9,1101,Abilene Chr,1144,Campbell,2019,29,0.0,0.0,A,0.0,...,14.0,22.0,26.0,4.0,17.0,11.0,11.0,14.0,0.0,23.0
17,1101,Abilene Chr,1146,Cent Arkansas,2019,82,0.0,0.0,A,0.0,...,20.0,17.0,24.0,10.0,27.0,18.0,12.0,10.0,6.0,21.0
18,1101,Abilene Chr,1146,Cent Arkansas,2019,117,0.0,0.0,H,0.0,...,24.0,3.0,3.0,4.0,29.0,18.0,9.0,8.0,0.0,13.0
31,1101,Abilene Chr,1176,Denver,2019,10,0.0,0.0,H,0.0,...,24.0,13.0,16.0,5.0,21.0,16.0,9.0,5.0,5.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217137,1466,North Alabama,1405,Toledo,2019,23,0.0,0.0,H,0.0,...,19.0,8.0,10.0,11.0,18.0,9.0,11.0,7.0,0.0,13.0
217138,1466,North Alabama,1407,Troy,2019,19,0.0,0.0,H,0.0,...,32.0,12.0,18.0,13.0,24.0,9.0,14.0,4.0,1.0,14.0
217141,1466,North Alabama,1412,UAB,2019,29,0.0,0.0,H,0.0,...,31.0,8.0,12.0,5.0,23.0,13.0,14.0,4.0,0.0,13.0
217144,1466,North Alabama,1422,UNC Greensboro,2019,40,0.0,0.0,H,0.0,...,24.0,4.0,6.0,10.0,33.0,7.0,14.0,3.0,6.0,21.0


In [34]:
team1_df = team1_df.sort_values(by=['Season', 'DayNum', 'TeamID', 'OppTeamID'])
team1_df.insert(0, 'GameId', range(1, len(team1_df)+1))

team2_df = team2_df.sort_values(by=['Season', 'DayNum', 'OppTeamID', 'TeamID'])
team2_df.insert(0, 'GameId', range(1, len(team2_df)+1))

team1_df['Home'] = np.where(team1_df['Loc'] == 'H', 1, 0)
team2_df['Home'] = np.where(team2_df['Loc'] == 'H', 0, 1)

In [35]:
team1 = team1_df.drop_duplicates(subset=['TeamID', 'TeamName', 'OppTeamID', 'OppTeamName', 'Season', 'DayNum',
       'ConfTourney', 'MMTourney', 'Loc', 'NumOT', 'Score', 'FGM', 'FGA',
       'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk',
       'PF'])
team2 = team2_df.drop_duplicates(subset=['TeamID', 'TeamName', 'OppTeamID', 'OppTeamName', 'Season', 'DayNum',
       'ConfTourney', 'MMTourney', 'Loc', 'NumOT', 'Score', 'FGM', 'FGA',
       'FGM3', 'FGA3', 'FTM', 'FTA', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk',
       'PF'])

In [36]:
games_df = pd.concat([team1_df, team2_df], axis=0)
games_df = games_df.sort_values(by=['Season', 'DayNum', 'TeamID', 'OppTeamID'])
games_df = games_df.sort_values(by=['GameId'])

games_df[['ConfTourney', 'MMTourney', 'TeamID', 'OppTeamID', 'Score']] = games_df[['ConfTourney', 'MMTourney', 'TeamID', 'OppTeamID', 'Score']].astype(int)
games_df = games_df.sort_values(by=['GameId'])

# games_df['Home'] = np.where(games_df['Loc'] == 'H', 1, 0)
games_df['HomeTeam'] = np.where(games_df['Home'] == 1, games_df['TeamName'], games_df['OppTeamName'])
games_df['AwayTeam'] = np.where(games_df['Home'] == 0, games_df['TeamName'], games_df['OppTeamName'])

In [37]:
games_df['NeutralSite'] = np.where(games_df['Loc'] == 'N', 1, 0)
games_df = games_df.rename(columns={'TeamName':'Team', 'FGM3':'3FGM',
                                    'FGA3':'3FGA', 'OR':'ORB', 'DR':'DRB', 'TO':'TOV',
                                    'Stl':'STL', 'Blk':'BLK', 'Ast':'AST', 'PF':'Fouls'})

games_df = games_df.drop(columns=['Season', 'OppTeamName', 'Loc'])
games_df

,GameId,OppTeamID,TeamID,Team,DayNum,ConfTourney,MMTourney,NumOT,Score,FGM,...,DRB,AST,TOV,STL,BLK,Fouls,Home,HomeTeam,AwayTeam,NeutralSite
164885,1,1380,1104,Alabama,1,0,0,0.0,82,27.0,...,29.0,17.0,20.0,3.0,6.0,24.0,1,Alabama,Southern Univ,0
164885,1,1104,1380,Southern Univ,1,0,0,0.0,62,21.0,...,20.0,8.0,15.0,7.0,2.0,28.0,0,Alabama,Southern Univ,0
163646,2,1378,1105,Alabama A&M,1,0,0,0.0,63,22.0,...,16.0,18.0,23.0,11.0,0.0,24.0,0,South Florida,Alabama A&M,0
163646,2,1105,1378,South Florida,1,0,0,0.0,80,30.0,...,22.0,20.0,20.0,11.0,4.0,15.0,1,South Florida,Alabama A&M,0
77892,3,1235,1106,Alabama St,1,0,0,0.0,53,20.0,...,23.0,11.0,20.0,4.0,3.0,17.0,0,Iowa St,Alabama St,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10970,11058,1438,1120,Auburn,152,0,1,0.0,62,21.0,...,24.0,9.0,5.0,3.0,3.0,12.0,1,Auburn,Virginia,1
201490,11059,1438,1403,Texas Tech,154,0,1,1.0,77,27.0,...,23.0,9.0,8.0,6.0,3.0,18.0,0,Virginia,Texas Tech,1
201490,11059,1403,1438,Virginia,154,0,1,1.0,85,27.0,...,28.0,15.0,11.0,4.0,3.0,15.0,1,Virginia,Texas Tech,1
180136,11060,1403,1438,Virginia,154,0,1,1.0,85,27.0,...,28.0,15.0,11.0,4.0,3.0,15.0,0,Texas Tech,Virginia,1


In [38]:
games_df = games_df.merge(team_conf_df[['TeamID', 'ConfAbbrev']].rename(columns={'ConfAbbrev':'Conf'}).drop_duplicates(subset=['TeamID'], keep='last'), on=['TeamID'], how='inner').sort_values('GameId')
games_df

,GameId,OppTeamID,TeamID,Team,DayNum,ConfTourney,MMTourney,NumOT,Score,FGM,...,AST,TOV,STL,BLK,Fouls,Home,HomeTeam,AwayTeam,NeutralSite,Conf
0,1,1380,1104,Alabama,1,0,0,0.0,82,27.0,...,17.0,20.0,3.0,6.0,24.0,1,Alabama,Southern Univ,0,sec
66,1,1104,1380,Southern Univ,1,0,0,0.0,62,21.0,...,8.0,15.0,7.0,2.0,28.0,0,Alabama,Southern Univ,0,swac
192,2,1105,1378,South Florida,1,0,0,0.0,80,30.0,...,20.0,20.0,11.0,4.0,15.0,1,South Florida,Alabama A&M,0,aac
128,2,1378,1105,Alabama A&M,1,0,0,0.0,63,22.0,...,18.0,23.0,11.0,0.0,24.0,0,South Florida,Alabama A&M,0,swac
254,3,1235,1106,Alabama St,1,0,0,0.0,53,20.0,...,11.0,20.0,4.0,3.0,17.0,0,Iowa St,Alabama St,0,swac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,11058,1438,1120,Auburn,152,0,1,0.0,62,21.0,...,9.0,5.0,3.0,3.0,12.0,1,Auburn,Virginia,1,sec
11722,11059,1403,1438,Virginia,154,0,1,1.0,85,27.0,...,15.0,11.0,4.0,3.0,15.0,1,Virginia,Texas Tech,1,acc
6594,11059,1438,1403,Texas Tech,154,0,1,1.0,77,27.0,...,9.0,8.0,6.0,3.0,18.0,0,Virginia,Texas Tech,1,big_twelve
11723,11060,1403,1438,Virginia,154,0,1,1.0,85,27.0,...,15.0,11.0,4.0,3.0,15.0,0,Texas Tech,Virginia,1,acc


In [39]:
games_df = games_df.merge(team_conf_df[['TeamID', 'ConfAbbrev']].rename(columns={'TeamID':'OppTeamID', 'ConfAbbrev':'OppConf'}).drop_duplicates(subset=['OppTeamID'], keep='last'), on=['OppTeamID'], how='inner').sort_values('GameId')
games_df

,GameId,OppTeamID,TeamID,Team,DayNum,ConfTourney,MMTourney,NumOT,Score,FGM,...,TOV,STL,BLK,Fouls,Home,HomeTeam,AwayTeam,NeutralSite,Conf,OppConf
0,1,1380,1104,Alabama,1,0,0,0.0,82,27.0,...,20.0,3.0,6.0,24.0,1,Alabama,Southern Univ,0,sec,swac
62,1,1104,1380,Southern Univ,1,0,0,0.0,62,21.0,...,15.0,7.0,2.0,28.0,0,Alabama,Southern Univ,0,swac,sec
192,2,1378,1105,Alabama A&M,1,0,0,0.0,63,22.0,...,23.0,11.0,0.0,24.0,0,South Florida,Alabama A&M,0,swac,aac
128,2,1105,1378,South Florida,1,0,0,0.0,80,30.0,...,20.0,11.0,4.0,15.0,1,South Florida,Alabama A&M,0,aac,swac
254,3,1235,1106,Alabama St,1,0,0,0.0,53,20.0,...,20.0,4.0,3.0,17.0,0,Iowa St,Alabama St,0,swac,big_twelve
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,11058,1120,1438,Virginia,152,0,1,0.0,63,25.0,...,8.0,1.0,9.0,12.0,0,Auburn,Virginia,1,acc,sec
11722,11059,1438,1403,Texas Tech,154,0,1,1.0,77,27.0,...,8.0,6.0,3.0,18.0,0,Virginia,Texas Tech,1,big_twelve,acc
6594,11059,1403,1438,Virginia,154,0,1,1.0,85,27.0,...,11.0,4.0,3.0,15.0,1,Virginia,Texas Tech,1,acc,big_twelve
11723,11060,1438,1403,Texas Tech,154,0,1,1.0,77,27.0,...,8.0,6.0,3.0,18.0,1,Texas Tech,Virginia,1,big_twelve,acc


In [40]:
games_df = games_df.drop_duplicates(subset=['Team', 'DayNum', 'Score', 'FGM', 'FGA', '3FGM', '3FGA',
       'FTM', 'FTA', 'ORB', 'DRB', 'AST', 'TOV', 'STL', 'BLK', 'Fouls', 'NeutralSite'])

In [41]:
games_df['GameId'] = [val for val in range(1, 1+len(games_df)//2) for _ in (0, 1)]

games_df[games_df['DayNum'] >= 135]

/var/folders/9y/flvy6tfd14v8rmvfr2c0fry40000gn/T/ipykernel_66211/3571553487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_df['GameId'] = [val for val in range(1, 1+len(games_df)//2) for _ in (0, 1)]


,GameId,OppTeamID,TeamID,Team,DayNum,ConfTourney,MMTourney,NumOT,Score,FGM,...,TOV,STL,BLK,Fouls,Home,HomeTeam,AwayTeam,NeutralSite,Conf,OppConf
564,5466,1113,1385,St John's,135,0,1,0.0,65,22.0,...,16.0,11.0,1.0,22.0,1,St John's,Arizona St,1,big_east,pac_twelve
7576,5466,1385,1113,Arizona St,135,0,1,0.0,74,22.0,...,21.0,5.0,2.0,18.0,0,St John's,Arizona St,1,pac_twelve,big_east
15586,5467,1300,1295,N Dakota St,135,0,1,0.0,78,25.0,...,6.0,0.0,3.0,14.0,0,NC Central,N Dakota St,1,summit,meac
9432,5467,1295,1300,NC Central,135,0,1,0.0,74,24.0,...,9.0,0.0,2.0,20.0,1,NC Central,N Dakota St,1,meac,summit
14344,5468,1101,1246,Kentucky,136,0,1,0.0,79,30.0,...,12.0,5.0,1.0,18.0,1,Kentucky,Abilene Chr,1,sec,wac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
958,5528,1120,1438,Virginia,152,0,1,0.0,63,25.0,...,8.0,1.0,9.0,12.0,1,Virginia,Auburn,1,acc,sec
7048,5529,1277,1403,Texas Tech,152,0,1,0.0,61,22.0,...,7.0,4.0,4.0,18.0,1,Texas Tech,Michigan St,1,big_twelve,big_ten
6592,5529,1403,1277,Michigan St,152,0,1,0.0,51,15.0,...,11.0,1.0,2.0,15.0,0,Texas Tech,Michigan St,1,big_ten,big_twelve
11722,5530,1438,1403,Texas Tech,154,0,1,1.0,77,27.0,...,8.0,6.0,3.0,18.0,0,Virginia,Texas Tech,1,big_twelve,acc


In [42]:
def to_int(lst):

  output_lst = []
  for item in lst:
    # print(item)
    item = item[1:]
    if item[-1].isalpha():
      item = item[:-1]
    output_lst.append(int(item))

  return output_lst

In [43]:
tourney_seeds_df['Seed'] = to_int(tourney_seeds_df['Seed'].tolist())

In [44]:
tourney_seeds_df = tourney_seeds_df[tourney_seeds_df['Season'] == year]

In [45]:
temp = games_df.merge(tourney_seeds_df[['Seed', 'TeamID']], on='TeamID', how='left')
temp.merge(tourney_seeds_df[['Seed', 'TeamID']].rename(columns={'Seed':'Opp_Seed', 'TeamID':'OppTeamID'}), on='OppTeamID', how='left')

,GameId,OppTeamID,TeamID,Team,DayNum,ConfTourney,MMTourney,NumOT,Score,FGM,...,BLK,Fouls,Home,HomeTeam,AwayTeam,NeutralSite,Conf,OppConf,Seed,Opp_Seed
0,1,1380,1104,Alabama,1,0,0,0.0,82,27.0,...,6.0,24.0,1,Alabama,Southern Univ,0,sec,swac,NaN,NaN
1,1,1104,1380,Southern Univ,1,0,0,0.0,62,21.0,...,2.0,28.0,0,Alabama,Southern Univ,0,swac,sec,NaN,NaN
2,2,1378,1105,Alabama A&M,1,0,0,0.0,63,22.0,...,0.0,24.0,0,South Florida,Alabama A&M,0,swac,aac,NaN,NaN
3,2,1105,1378,South Florida,1,0,0,0.0,80,30.0,...,4.0,15.0,1,South Florida,Alabama A&M,0,aac,swac,NaN,NaN
4,3,1235,1106,Alabama St,1,0,0,0.0,53,20.0,...,3.0,17.0,0,Iowa St,Alabama St,0,swac,big_twelve,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11055,5528,1120,1438,Virginia,152,0,1,0.0,63,25.0,...,9.0,12.0,1,Virginia,Auburn,1,acc,sec,1.0,5.0
11056,5529,1277,1403,Texas Tech,152,0,1,0.0,61,22.0,...,4.0,18.0,1,Texas Tech,Michigan St,1,big_twelve,big_ten,3.0,2.0
11057,5529,1403,1277,Michigan St,152,0,1,0.0,51,15.0,...,2.0,15.0,0,Texas Tech,Michigan St,1,big_ten,big_twelve,2.0,3.0
11058,5530,1438,1403,Texas Tech,154,0,1,1.0,77,27.0,...,3.0,18.0,0,Virginia,Texas Tech,1,big_twelve,acc,3.0,1.0


In [46]:
games_df = temp

In [47]:
games_df.columns

Index(['GameId', 'OppTeamID', 'TeamID', 'Team', 'DayNum', 'ConfTourney',
       'MMTourney', 'NumOT', 'Score', 'FGM', 'FGA', '3FGM', '3FGA', 'FTM',
       'FTA', 'ORB', 'DRB', 'AST', 'TOV', 'STL', 'BLK', 'Fouls', 'Home',
       'HomeTeam', 'AwayTeam', 'NeutralSite', 'Conf', 'OppConf', 'Seed'],
      dtype='object')

### Save Wrangling Results

Now that we are done with the wrangling for ML, let's save the results as zipped csv files before preceeding to EDA.

In [48]:
compression_opts = dict(method='zip', archive_name='ncaam' + str(year) + '.csv')
games_df.to_csv('ncaam' + str(year) + '.zip', index=False, compression=compression_opts)